In [2]:
from sympy import sin, cos, symbols, lambdify
import numpy as np
x = symbols('x')
expr = sin(x) + cos(x)
expr
f = lambdify(x, expr, 'numpy')
a = np.array([1, 2])
f(a)
# type(f(a))

array([1.38177329, 0.49315059])

In [1]:
from gymnax.environments import environment, spaces
import numpy as np
import jax
import chex
action_space = spaces.Box(-1.0,1.0,shape=(2,3),dtype=np.float32)
rng = jax.random.PRNGKey(0)
action_space.sample(rng=rng)


/home/bianzx/miniconda3/envs/pytorch3d/lib/python3.9/site-packages/flax/core/frozen_dict.py:169: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(


Array([[ 0.14900088, -0.80062795,  0.48393178],
       [ 0.78835654,  0.19313312, -0.09349632]], dtype=float32)

In [7]:

from quadjax.dynamics.dataclass import Action
import numpy as np
thrust = 1.0
tau = 0.5
env_action = Action(thrust=np.array([thrust, thrust]), tau = np.array([tau, tau]))
env_action


Action(thrust=array([1., 1.]), tau=array([0.5, 0.5]))

In [ ]:
import sympy as sp
from sympy.physics.mechanics import (
    dynamicsymbols,
    ReferenceFrame,
    Point,
    Particle,
    RigidBody,
    inertia,
)

# Define symbols
t = sp.Symbol("t")  # time
m = sp.Symbol("m", positive=True)  # mass of the quadrotor
I = sp.Symbol("I", positive=True)  # moment of inertia
g = sp.Symbol("g", positive=True)  # gravitational acceleration
l = sp.Symbol("l", positive=True)  # length of the rod
# mass of the object attached to the rod
mo = sp.Symbol("mo", positive=True)
thrust = sp.Function("thrust")(t)  # thrust force
tau = sp.Function("tau")(t)  # torque
f_rope = sp.Symbol("f_rope")  # force in the rope
# y displacement of the hook from the quadrotor center
delta_yh = sp.Symbol("delta_yh")
# z displacement of the hook from the quadrotor center
delta_zh = sp.Symbol("delta_zh")
params = [m, I, g, l, mo, delta_yh, delta_zh]
action = [thrust, tau]


y, z, theta, phi = dynamicsymbols("y z theta phi")
y_dot, z_dot, theta_dot, phi_dot = sp.diff(y, t), sp.diff(
    z, t), sp.diff(theta, t), sp.diff(phi, t)
y_ddot, z_ddot, theta_ddot, phi_ddot = sp.diff(y_dot, t), sp.diff(
    z_dot, t), sp.diff(theta_dot, t), sp.diff(phi_dot, t)
y_dot_val, z_dot_val, theta_dot_val, phi_dot_val = sp.symbols(
    "y_dot_val z_dot_val theta_dot_val phi_dot_val")
y_ddot_val, z_ddot_val, theta_ddot_val, phi_ddot_val = sp.symbols(
    "y_ddot_val z_ddot_val theta_ddot_val phi_ddot_val")
states = [y, z, theta, phi, y_dot, z_dot, theta_dot, phi_dot]
states_val = [y, z, theta, phi, y_dot_val,
                z_dot_val, theta_dot_val, phi_dot_val]
states_dot = [y_ddot, z_ddot, theta_ddot, phi_ddot, f_rope]
states_dot_val = [y_ddot_val, z_ddot_val,
                    theta_ddot_val, phi_ddot_val, f_rope]

states_val = [y, z, theta, phi, y_dot_val,
                  z_dot_val, theta_dot_val, phi_dot_val]

# Solve for the acceleration
A_taut_dyn = sp.zeros(5, 5)
b_taut_dyn = sp.zeros(5, 1)

# lambda A_taut_dyn
A_taut_dyn_func = sp.lambdify(
    params + states_val + action, A_taut_dyn, "jax")
b_taut_dyn_func = sp.lambdify(
    params + states_val + action, b_taut_dyn, "jax")

In [1]:
from sympy import symbols
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame
from sympy.physics.mechanics import RigidBody, Particle, Point, outer
from sympy.physics.mechanics import linear_momentum, angular_momentum
from sympy.physics.vector import init_vprinting
init_vprinting(pretty_print=False)
m, M, l1 = symbols('m M l1')
q1d = dynamicsymbols('q1d')
N = ReferenceFrame('N')
O = Point('O')
O.set_vel(N, 0 * N.x)
Ac = O.locatenew('Ac', l1 * N.x)
P = Ac.locatenew('P', l1 * N.x)
a = ReferenceFrame('a')
a.set_ang_vel(N, q1d * N.z)
Ac.v2pt_theory(O, N, a), P.v2pt_theory(O, N, a)

(l1*q1d*N.y, 2*l1*q1d*N.y)

In [2]:
import sympy as sp
from sympy.physics.mechanics import (
dynamicsymbols,
ReferenceFrame,
Point,
Particle,
RigidBody,
inertia,
)
from jax import numpy as jnp
from functools import partial
import sys
sys.path.append("/home/bianzx/jax/quadjax/")
from quadjax.dynamics.utils import angle_normalize
from quadjax.dynamics import geom
from quadjax.dynamics.dataclass import EnvParams, EnvState, Action, EnvParams3D, EnvState3D, Action3D
from typing import Tuple
from jax import jit
import numpy as np


# def get_taut_dynamics():

# Define symbols
t = sp.Symbol("t")  # time
m = sp.Symbol("m", positive=True)  # mass of the quadrotor
I = sp.Symbol("I", positive=True)  # moment of inertia
g = sp.Symbol("g", positive=True)  # gravitational acceleration
l = sp.Symbol("l", positive=True)  # length of the rod
# mass of the object attached to the rod
mo = sp.Symbol("mo", positive=True)
thrust = sp.Function("thrust")(t)  # thrust force
tau = sp.Function("tau")(t)  # torque
f_rope = sp.Symbol("f_rope")  # force in the rope
thrust2 = sp.Function("thrust2")(t)  # thrust force
tau2 = sp.Function("tau2")(t)  # torque
f_rope2 = sp.Symbol("f_rope2")  # force in the rope
# y displacement of the hook from the quadrotor center
delta_yh = sp.Symbol("delta_yh")
# z displacement of the hook from the quadrotor center
delta_zh = sp.Symbol("delta_zh")
# y displacement of the hook from the quadrotor center
delta_yh2 = sp.Symbol("delta_yh2")
# z displacement of the hook from the quadrotor center
delta_zh2 = sp.Symbol("delta_zh2")
# TODO...
params = [m, I, g, l, mo, delta_yh, delta_zh, delta_yh2, delta_zh2]
# action = [thrust, tau]
action = [thrust, tau, thrust2, tau2]

# Define state variables and their derivatives
y, z, theta, phi = dynamicsymbols("y z theta phi")
y_dot, z_dot, theta_dot, phi_dot = sp.diff(y, t), sp.diff(
    z, t), sp.diff(theta, t), sp.diff(phi, t)
y_ddot, z_ddot, theta_ddot, phi_ddot = sp.diff(y_dot, t), sp.diff(
    z_dot, t), sp.diff(theta_dot, t), sp.diff(phi_dot, t)
y_dot_val, z_dot_val, theta_dot_val, phi_dot_val = sp.symbols(
    "y_dot_val z_dot_val theta_dot_val phi_dot_val")
y_ddot_val, z_ddot_val, theta_ddot_val, phi_ddot_val = sp.symbols(
    "y_ddot_val z_ddot_val theta_ddot_val phi_ddot_val")
# states = [y, z, theta, phi, y_dot, z_dot, theta_dot, phi_dot]
# states_val = [y, z, theta, phi, y_dot_val,
#               z_dot_val, theta_dot_val, phi_dot_val]
# states_dot = [y_ddot, z_ddot, theta_ddot, phi_ddot, f_rope]
# states_dot_val = [y_ddot_val, z_ddot_val,
#                   theta_ddot_val, phi_ddot_val, f_rope]

y2, z2, theta2, phi2 = dynamicsymbols("y2 z2 theta2 phi2")
y2_dot, z2_dot, theta2_dot, phi2_dot = sp.diff(y2, t), sp.diff(
    z2, t), sp.diff(theta2, t), sp.diff(phi2, t)
y2_ddot, z2_ddot, theta2_ddot, phi2_ddot = sp.diff(y2_dot, t), sp.diff(
    z2_dot, t), sp.diff(theta2_dot, t), sp.diff(phi2_dot, t)
y2_dot_val, z2_dot_val, theta2_dot_val, phi2_dot_val = sp.symbols(
    "y2_dot_val z2_dot_val theta2_dot_val phi2_dot_val")
y2_ddot_val, z2_ddot_val, theta2_ddot_val, phi2_ddot_val = sp.symbols(
    "y2_ddot_val z2_ddot_val theta2_ddot_val phi2_ddot_val")

states = [y, z, theta, phi, y_dot, z_dot, theta_dot, phi_dot,
        y2, z2, theta2, phi2, y2_dot, z2_dot, theta2_dot, phi2_dot]
states_val = [y, z, theta, phi, y_dot_val,
                z_dot_val, theta_dot_val, phi_dot_val, 
                y2, z2, theta2, phi2, y2_dot_val, z2_dot_val, theta2_dot_val, phi2_dot_val]
states_dot = [y_ddot, z_ddot, theta_ddot, phi_ddot, f_rope, y2_ddot, z2_ddot, theta2_ddot, phi2_ddot, f_rope2]
states_dot_val = [y_ddot_val, z_ddot_val,
                    theta_ddot_val, phi_ddot_val, f_rope, 
                    y2_ddot_val, z2_ddot_val,
                    theta2_ddot_val, phi2_ddot_val, f_rope2]

# intermediate variables
delta_yh_global = delta_yh * sp.cos(theta) - delta_zh * sp.sin(theta)
delta_zh_global = delta_yh * sp.sin(theta) + delta_zh * sp.cos(theta)
f_rope_y = f_rope * sp.sin(theta+phi)
f_rope_z = -f_rope * sp.cos(theta+phi)
y_hook = y + delta_yh_global
z_hook = z + delta_zh_global
y_hook_dot = sp.diff(y_hook, t)
z_hook_dot = sp.diff(z_hook, t)
y_obj = y_hook + l * sp.sin(theta+phi)
z_obj = z_hook - l * sp.cos(theta+phi)
y_obj_dot = sp.diff(y_obj, t)
z_obj_dot = sp.diff(z_obj, t)
y_obj_ddot = sp.diff(y_obj_dot, t)
z_obj_ddot = sp.diff(z_obj_dot, t)
obses = [y_obj, z_obj, y_obj_dot, z_obj_dot,
            y_obj_ddot, z_obj_ddot, f_rope_y, f_rope_z]

delta_yh2_global = delta_yh2 * sp.cos(theta2) - delta_zh * sp.sin(theta2)
delta_zh2_global = delta_yh2 * sp.sin(theta2) + delta_zh * sp.cos(theta2)
f_rope2_y = f_rope2 * sp.sin(theta2+phi2)
f_rope2_z = -f_rope2 * sp.cos(theta2+phi2)
y_hook2 = y2 + delta_yh2_global
z_hook2 = z2 + delta_zh2_global
y_hook2_dot = sp.diff(y_hook2, t)
z_hook2_dot = sp.diff(z_hook2, t)
# obses = [y_obj, z_obj, y_obj_dot, z_obj_dot,
#          y_obj_ddot, z_obj_ddot, f_rope_y, f_rope_z]


# Define inertial reference frame
N = ReferenceFrame("N")
N_origin = Point('N_origin')
A = N.orientnew("A", "Axis", [theta, N.x])
B = A.orientnew("B", "Axis", [phi, A.x])

# Define point
drone = Point("drone")
drone.set_pos(N_origin, y * N.y + z * N.z)
hook = drone.locatenew("hook", delta_yh * A.y + delta_zh * A.z)
drone2 = Point("drone2")
drone2.set_pos(N_origin, y2 * N.y + z2 * N.z)
hook2 = drone2.locatenew("hook2", delta_yh2 * A.y + delta_zh2 * A.z)
obj = hook.locatenew("obj", -l * B.z)
drone.set_vel(N, y_dot * N.y + z_dot * N.z)
drone2.set_vel(N, y2_dot * N.y + z2_dot * N.z)

# Inertia
inertia_quadrotor = inertia(N, I, 0, 0)
quadrotor = RigidBody("quadrotor", drone, A, m, (inertia_quadrotor, drone))
quadrotor2 = RigidBody("quadrotor2", drone2, A, m, (inertia_quadrotor, drone2))
obj_particle = Particle("obj_particle", obj, mo)

# Newton's law
# 8 equations
eq_quad_y = -thrust * sp.sin(theta) + f_rope_y - m * y_ddot
eq_quad_z = thrust * sp.cos(theta) + f_rope_z - m * g - m * z_ddot
eq_quad_theta = tau + delta_yh_global * f_rope_z - \
    delta_zh_global * f_rope_y - I * theta_ddot
# eq_obj_y = -f_rope_y - mo * y_obj_ddot
# eq_obj_z = -f_rope_z - mo * g - mo * z_obj_ddot

eq_quad_y2 = -thrust2 * sp.sin(theta2) + f_rope2_y - m * y2_ddot
eq_quad_z2 = thrust2 * sp.cos(theta2) + f_rope2_z - m * g - m * z2_ddot
eq_quad_theta2 = tau2 + delta_yh2_global * f_rope2_z - \
    delta_zh2_global * f_rope2_y - I * theta2_ddot
eq_obj_y = -f_rope_y -f_rope2_y - mo * y_obj_ddot
eq_obj_z = -f_rope_z -f_rope2_z - mo * g - mo * z_obj_ddot
eq_quad1_obj = (y-y_obj)*(y_dot-y_obj_dot)+(z-z_obj)*(z_dot-z_obj_dot)
eq_quad2_obj = (y2-y_obj)*(y2_dot-y_obj_dot)+(z2-z_obj)*(z2_dot-z_obj_dot)

# TODO...
eqs = [eq_quad_y, eq_quad_z, eq_quad_theta, eq_obj_y, eq_obj_z, eq_quad_y2, eq_quad_z2, eq_quad_theta2, eq_quad1_obj, eq_quad2_obj]
eqs = [eq.expand() for eq in eqs]
eqs = [eq.subs([(states_dot[i], states_dot_val[i])
                for i in range(len(states_dot))]) for eq in eqs]
eqs = [eq.subs([(states[i], states_val[i])
                for i in range(len(states))]) for eq in eqs]
eqs


[f_rope*sin(phi(t) + theta(t)) - m*y_ddot_val - thrust(t)*sin(theta(t)),
 -f_rope*cos(phi(t) + theta(t)) - g*m - m*z_ddot_val + thrust(t)*cos(theta(t)),
 -I*theta_ddot_val - delta_yh*f_rope*sin(phi(t) + theta(t))*sin(theta(t)) - delta_yh*f_rope*cos(phi(t) + theta(t))*cos(theta(t)) - delta_zh*f_rope*sin(phi(t) + theta(t))*cos(theta(t)) + delta_zh*f_rope*sin(theta(t))*cos(phi(t) + theta(t)) + tau(t),
 delta_yh*mo*theta_ddot_val*sin(theta(t)) + delta_yh*mo*theta_dot_val**2*cos(theta(t)) + delta_zh*mo*theta_ddot_val*cos(theta(t)) - delta_zh*mo*theta_dot_val**2*sin(theta(t)) - f_rope*sin(phi(t) + theta(t)) - f_rope2*sin(phi2(t) + theta2(t)) - l*mo*phi_ddot_val*cos(phi(t) + theta(t)) + l*mo*phi_dot_val**2*sin(phi(t) + theta(t)) + 2*l*mo*phi_dot_val*theta_dot_val*sin(phi(t) + theta(t)) - l*mo*theta_ddot_val*cos(phi(t) + theta(t)) + l*mo*theta_dot_val**2*sin(phi(t) + theta(t)) - mo*y_ddot_val,
 -delta_yh*mo*theta_ddot_val*cos(theta(t)) + delta_yh*mo*theta_dot_val**2*sin(theta(t)) + delta_zh*mo

In [ ]:

# Solve for the acceleration
A_taut_dyn = sp.zeros(8, 8)
b_taut_dyn = sp.zeros(8, 1)
for i in range(8):
    for j in range(8):
        A_taut_dyn[i, j] = eqs[i].coeff(states_dot_val[j])
    b_taut_dyn[i] = -eqs[i].subs([(states_dot_val[j], 0)
                                    for j in range(8)])
# lambda A_taut_dyn
A_taut_dyn_func = sp.lambdify(
    params + states_val + action, A_taut_dyn, "jax")
b_taut_dyn_func = sp.lambdify(
    params + states_val + action, b_taut_dyn, "jax")
params + states_val + action
A_taut_dyn

In [10]:
import sympy as sp
from sympy.physics.mechanics import (
dynamicsymbols,
ReferenceFrame,
Point,
Particle,
RigidBody,
inertia,
)
from jax import numpy as jnp
from functools import partial
import sys
sys.path.append("/home/bianzx/jax/quadjax/")
from quadjax.dynamics.utils import angle_normalize
from quadjax.dynamics import geom
from quadjax.dynamics.dataclass import EnvParams, EnvState, Action, EnvParams3D, EnvState3D, Action3D
from typing import Tuple
from jax import jit
import numpy as np

# Define symbols
t = sp.Symbol("t")  # time
m = sp.Symbol("m", positive=True)  # mass of the quadrotor
I = sp.Symbol("I", positive=True)  # moment of inertia
g = sp.Symbol("g", positive=True)  # gravitational acceleration
l = sp.Symbol("l", positive=True)  # length of the rod
# mass of the object attached to the rod
mo = sp.Symbol("mo", positive=True)
thrust = sp.Function("thrust")(t)  # thrust force
tau = sp.Function("tau")(t)  # torque
f_rope = sp.Symbol("f_rope")  # force in the rope
# y displacement of the hook from the quadrotor center
delta_yh = sp.Symbol("delta_yh")
# z displacement of the hook from the quadrotor center
delta_zh = sp.Symbol("delta_zh")
params = [m, I, g, l, mo, delta_yh, delta_zh]
action = [thrust, tau]

# Define state variables and their derivatives
y, z, theta, phi = dynamicsymbols("y z theta phi")
y_dot, z_dot, theta_dot, phi_dot = sp.diff(y, t), sp.diff(
    z, t), sp.diff(theta, t), sp.diff(phi, t)
y_ddot, z_ddot, theta_ddot, phi_ddot = sp.diff(y_dot, t), sp.diff(
    z_dot, t), sp.diff(theta_dot, t), sp.diff(phi_dot, t)
y_dot_val, z_dot_val, theta_dot_val, phi_dot_val = sp.symbols(
    "y_dot_val z_dot_val theta_dot_val phi_dot_val")
y_ddot_val, z_ddot_val, theta_ddot_val, phi_ddot_val = sp.symbols(
    "y_ddot_val z_ddot_val theta_ddot_val phi_ddot_val")
states = [y, z, theta, phi, y_dot, z_dot, theta_dot, phi_dot]
states_val = [y, z, theta, phi, y_dot_val,
                z_dot_val, theta_dot_val, phi_dot_val]
states_dot = [y_ddot, z_ddot, theta_ddot, phi_ddot, f_rope]
states_dot_val = [y_ddot_val, z_ddot_val,
                    theta_ddot_val, phi_ddot_val, f_rope]

# intermediate variables
delta_yh_global = delta_yh * sp.cos(theta) - delta_zh * sp.sin(theta)
delta_zh_global = delta_yh * sp.sin(theta) + delta_zh * sp.cos(theta)
f_rope_y = f_rope * sp.sin(theta+phi)
f_rope_z = -f_rope * sp.cos(theta+phi)
y_hook = y + delta_yh_global
z_hook = z + delta_zh_global
y_hook_dot = sp.diff(y_hook, t)
z_hook_dot = sp.diff(z_hook, t)
y_obj = y_hook + l * sp.sin(theta+phi)
z_obj = z_hook - l * sp.cos(theta+phi)
y_obj_dot = sp.diff(y_obj, t)
z_obj_dot = sp.diff(z_obj, t)
y_obj_ddot = sp.diff(y_obj_dot, t)
z_obj_ddot = sp.diff(z_obj_dot, t)
obses = [y_obj, z_obj, y_obj_dot, z_obj_dot,
            y_obj_ddot, z_obj_ddot, f_rope_y, f_rope_z]

# Define inertial reference frame
N = ReferenceFrame("N")
N_origin = Point('N_origin')
A = N.orientnew("A", "Axis", [theta, N.x])
B = A.orientnew("B", "Axis", [phi, A.x])

# Define point
drone = Point("drone")
drone.set_pos(N_origin, y * N.y + z * N.z)
hook = drone.locatenew("hook", delta_yh * A.y + delta_zh * A.z)
obj = hook.locatenew("obj", -l * B.z)
drone.set_vel(N, y_dot * N.y + z_dot * N.z)

# Inertia
inertia_quadrotor = inertia(N, I, 0, 0)
quadrotor = RigidBody("quadrotor", drone, A, m, (inertia_quadrotor, drone))
obj_particle = Particle("obj_particle", obj, mo)

# Newton's law
eq_quad_y = -thrust * sp.sin(theta) + f_rope_y - m * y_ddot
eq_quad_z = thrust * sp.cos(theta) + f_rope_z - m * g - m * z_ddot
eq_quad_theta = tau + delta_yh_global * f_rope_z - \
    delta_zh_global * f_rope_y - I * theta_ddot
eq_obj_y = -f_rope_y - mo * y_obj_ddot
eq_obj_z = -f_rope_z - mo * g - mo * z_obj_ddot

eqs = [eq_quad_y, eq_quad_z, eq_quad_theta, eq_obj_y, eq_obj_z]
eqs = [eq.expand() for eq in eqs]
eqs = [eq.subs([(states_dot[i], states_dot_val[i])
                for i in range(len(states_dot))]) for eq in eqs]
eqs = [eq.subs([(states[i], states_val[i])
                for i in range(len(states))]) for eq in eqs]
# Solve for the acceleration
A_taut_dyn = sp.zeros(5, 5)
b_taut_dyn = sp.zeros(5, 1)
for i in range(5):
    for j in range(5):
        A_taut_dyn[i, j] = eqs[i].coeff(states_dot_val[j])
    b_taut_dyn[i] = -eqs[i].subs([(states_dot_val[j], 0)
                                    for j in range(5)])
A_taut_dyn

Matrix([
[ -m,   0,                                                                                    0,                            0,                                                                                                                                                                         sin(phi(t) + theta(t))],
[  0,  -m,                                                                                    0,                            0,                                                                                                                                                                        -cos(phi(t) + theta(t))],
[  0,   0,                                                                                   -I,                            0, -delta_yh*sin(phi(t) + theta(t))*sin(theta(t)) - delta_yh*cos(phi(t) + theta(t))*cos(theta(t)) - delta_zh*sin(phi(t) + theta(t))*cos(theta(t)) + delta_zh*sin(theta(t))*cos(phi(t) + theta(t))],
[-mo,   0,  delta_yh*mo*sin(the

In [ ]:
# lambda A_taut_dyn
A_taut_dyn_func = sp.lambdify(
    params + states_val + action, A_taut_dyn, "jax")
b_taut_dyn_func = sp.lambdify(
    params + states_val + action, b_taut_dyn, "jax")
A_taut_dyn_func

env_params = EnvParams(m=1.0,I=0.01,g=0.05,l=0.1,mo=1.0,delta_yh=0.5,delta_zh=0.1,delta_yh2=0.5,delta_zh2=0.1)
params = [env_params.m, env_params.I, env_params.g, env_params.l,
                  env_params.mo, env_params.delta_yh, env_params.delta_zh,env_params.delta_yh2, env_params.delta_zh2]
# params
jax.debug.print("params: {}", params)
A_taut_dyn_func, b_taut_dyn_func

In [ ]:
# Solve for the acceleration
A_taut_dyn = sp.zeros(5, 5)
b_taut_dyn = sp.zeros(5, 1)
for i in range(5):
    for j in range(5):
        A_taut_dyn[i, j] = eqs[i].coeff(states_dot_val[j])
    b_taut_dyn[i] = -eqs[i].subs([(states_dot_val[j], 0)
                                    for j in range(5)])
# lambda A_taut_dyn
A_taut_dyn_func = sp.lambdify(
    params + states_val + action, A_taut_dyn, "jax")
b_taut_dyn_func = sp.lambdify(
    params + states_val + action, b_taut_dyn, "jax")
A_taut_dyn_func

env_params = EnvParams(m=1.0,I=0.01,g=0.05,l=0.1,mo=1.0,delta_yh=0.5,delta_zh=0.1,delta_yh2=0.5,delta_zh2=0.1)
params = [env_params.m, env_params.I, env_params.g, env_params.l,
                  env_params.mo, env_params.delta_yh, env_params.delta_zh,env_params.delta_yh2, env_params.delta_zh2]
# params
jax.debug.print("params: {}", params)
A_taut_dyn_func, b_taut_dyn_func

In [ ]:
eqs = [eq.subs([(states_dot[i], states_dot_val[i])
                for i in range(len(states_dot))]) for eq in eqs]
eqs = [eq.subs([(states[i], states_val[i])
                for i in range(len(states))]) for eq in eqs]
# Solve for the acceleration
A_taut_dyn = sp.zeros(5, 5)
b_taut_dyn = sp.zeros(5, 1)
for i in range(5):
    for j in range(5):
        A_taut_dyn[i, j] = eqs[i].coeff(states_dot_val[j])
    b_taut_dyn[i] = -eqs[i].subs([(states_dot_val[j], 0)
                                    for j in range(5)])
# lambda A_taut_dyn
A_taut_dyn_func = sp.lambdify(
    params + states_val + action, A_taut_dyn, "jax")
b_taut_dyn_func = sp.lambdify(
    params + states_val + action, b_taut_dyn, "jax")
A_taut_dyn_func

env_params = EnvParams(m=1.0,I=0.01,g=0.05,l=0.1,mo=1.0,delta_yh=0.5,delta_zh=0.1,delta_yh2=0.5,delta_zh2=0.1)
params = [env_params.m, env_params.I, env_params.g, env_params.l,
                  env_params.mo, env_params.delta_yh, env_params.delta_zh,env_params.delta_yh2, env_params.delta_zh2]
# params
jax.debug.print("params: {}", params)
A_taut_dyn_func, b_taut_dyn_func

In [3]:
import sympy
x, y, z = sympy.symbols('x:z')
f = sympy.exp(x + y) - sympy.sqrt(z)
f.free_symbols

{x, y, z}

In [38]:

a = Action(3.0,3.0)
b = Action(4.0,4.0)
x = (a,b)
x[0].thrust,x[0].tau,x[1].thrust,x[1].tau


(3.0, 3.0, 4.0, 4.0)

In [12]:
from gymnax.environments import environment, spaces
import numpy as np
import jax
import chex
# action_space = spaces.Box(-1.0,1.0,shape=(2,3),dtype=np.float32)
action_space= spaces.Box(
            low=-1.0,
            high=1.0,
            shape=(4,),
            dtype=jnp.float32,
        )
rng = jax.random.PRNGKey(0)
action_space.sample(rng=rng)
# action_space
random_policy = lambda obs, rng: action_space(EnvParams()).sample(rng)

random_policy

<function __main__.<lambda>(obs, rng)>

In [1]:
import jax
import jax.numpy as jnp

@jax.jit
def f(x):
  jax.debug.print("🤯 {x} 🤯", x=x)
  y = jnp.sin(x)
  jax.debug.print("🤯 {y} 🤯", y=y)
  return y

f(2.)
# Prints:
# 🤯 2.0 🤯
# 🤯 0.9092974662780762 🤯

🤯 0.9092974662780762 🤯
🤯 2.0 🤯


Array(0.90929747, dtype=float32, weak_type=True)

In [3]:
import jax
import jax.numpy as jnp
from jax import lax
from gymnax.environments import environment, spaces
from typing import Tuple, Optional
import chex
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from functools import partial
from dataclasses import dataclass as pydataclass
import tyro
import sys
sys.path.append("/home/bianzx/jax/quadjax/")
from quadjax.dynamics.utils import angle_normalize, get_hit_penalty
from quadjax.dynamics.dataclass import EnvParams, EnvState, Action
# from quadjax.dynamics.taut_dual import get_taut_dynamics
# from quadjax.dynamics.loose import get_loose_dynamics
from quadjax.dynamics.loose_dual import get_loose_dynamics
from quadjax.dynamics.trans import get_dynamic_transfer

import sympy as sp
from sympy.physics.mechanics import (
    dynamicsymbols,
    ReferenceFrame,
    Point,
    Particle,
    RigidBody,
    inertia,
)
import jax
from jax import numpy as jnp
from functools import partial

from quadjax.dynamics.utils import angle_normalize
from quadjax.dynamics import geom
from quadjax.dynamics.dataclass import EnvParams, EnvState, Action, EnvParams3D, EnvState3D, Action3D
from typing import Tuple
from jax import jit
import numpy as np

# Enable debug mode with print statements
jax.config.update("jax_debug_nans", True)

# def get_taut_dynamics():

# Define symbols
t = sp.Symbol("t")  # time
m = sp.Symbol("m", positive=True)  # mass of the quadrotor
I = sp.Symbol("I", positive=True)  # moment of inertia
g = sp.Symbol("g", positive=True)  # gravitational acceleration
l = sp.Symbol("l", positive=True)  # length of the rod
# mass of the object attached to the rod
mo = sp.Symbol("mo", positive=True)
thrust = sp.Function("thrust")(t)  # thrust force
tau = sp.Function("tau")(t)  # torque
f_rope = sp.Symbol("f_rope")  # force in the rope
thrust2 = sp.Function("thrust2")(t)  # thrust force
tau2 = sp.Function("tau2")(t)  # torque
f_rope2 = sp.Symbol("f_rope2")  # force in the rope
# y displacement of the hook from the quadrotor center
delta_yh = sp.Symbol("delta_yh")
# z displacement of the hook from the quadrotor center
delta_zh = sp.Symbol("delta_zh")
# y displacement of the hook from the quadrotor center
delta_yh2 = sp.Symbol("delta_yh2")
# z displacement of the hook from the quadrotor center
delta_zh2 = sp.Symbol("delta_zh2")
# TODO...
params = [m, I, g, l, mo, delta_yh, delta_zh, delta_yh2, delta_zh2]
# action = [thrust, tau]
action = [thrust, tau, thrust2, tau2]

# Define state variables and their derivatives
y, z, theta, phi = dynamicsymbols("y z theta phi")
y_dot, z_dot, theta_dot, phi_dot = sp.diff(y, t), sp.diff(
    z, t), sp.diff(theta, t), sp.diff(phi, t)
y_ddot, z_ddot, theta_ddot, phi_ddot = sp.diff(y_dot, t), sp.diff(
    z_dot, t), sp.diff(theta_dot, t), sp.diff(phi_dot, t)
y_dot_val, z_dot_val, theta_dot_val, phi_dot_val = sp.symbols(
    "y_dot_val z_dot_val theta_dot_val phi_dot_val")
y_ddot_val, z_ddot_val, theta_ddot_val, phi_ddot_val = sp.symbols(
    "y_ddot_val z_ddot_val theta_ddot_val phi_ddot_val")
# states = [y, z, theta, phi, y_dot, z_dot, theta_dot, phi_dot]
# states_val = [y, z, theta, phi, y_dot_val,
#               z_dot_val, theta_dot_val, phi_dot_val]
# states_dot = [y_ddot, z_ddot, theta_ddot, phi_ddot, f_rope]
# states_dot_val = [y_ddot_val, z_ddot_val,
#                   theta_ddot_val, phi_ddot_val, f_rope]

y2, z2, theta2, phi2 = dynamicsymbols("y2 z2 theta2 phi2")
y2_dot, z2_dot, theta2_dot, phi2_dot = sp.diff(y2, t), sp.diff(
    z2, t), sp.diff(theta2, t), sp.diff(phi2, t)
y2_ddot, z2_ddot, theta2_ddot, phi2_ddot = sp.diff(y2_dot, t), sp.diff(
    z2_dot, t), sp.diff(theta2_dot, t), sp.diff(phi2_dot, t)
y2_dot_val, z2_dot_val, theta2_dot_val, phi2_dot_val = sp.symbols(
    "y2_dot_val z2_dot_val theta2_dot_val phi2_dot_val")
y2_ddot_val, z2_ddot_val, theta2_ddot_val, phi2_ddot_val = sp.symbols(
    "y2_ddot_val z2_ddot_val theta2_ddot_val phi2_ddot_val")

states = [y, z, theta, phi, y_dot, z_dot, theta_dot, phi_dot,
        y2, z2, theta2, phi2, y2_dot, z2_dot, theta2_dot, phi2_dot]
states_val = [y, z, theta, phi, y_dot_val,
                z_dot_val, theta_dot_val, phi_dot_val, 
                y2, z2, theta2, phi2, y2_dot_val, z2_dot_val, theta2_dot_val, phi2_dot_val]
states_dot = [y_ddot, z_ddot, theta_ddot, phi_ddot, f_rope, y2_ddot, z2_ddot, theta2_ddot, phi2_ddot, f_rope2]
states_dot_val = [y_ddot_val, z_ddot_val,
                    theta_ddot_val, phi_ddot_val, f_rope, 
                    y2_ddot_val, z2_ddot_val,
                    theta2_ddot_val, phi2_ddot_val, f_rope2]

# intermediate variables
delta_yh_global = delta_yh * sp.cos(theta) - delta_zh * sp.sin(theta)
delta_zh_global = delta_yh * sp.sin(theta) + delta_zh * sp.cos(theta)
f_rope_y = f_rope * sp.sin(theta+phi)
f_rope_z = -f_rope * sp.cos(theta+phi)
y_hook = y + delta_yh_global
z_hook = z + delta_zh_global
y_hook_dot = sp.diff(y_hook, t)
z_hook_dot = sp.diff(z_hook, t)
y_obj = y_hook + l * sp.sin(theta+phi)
z_obj = z_hook - l * sp.cos(theta+phi)
y_obj_dot = sp.diff(y_obj, t)
z_obj_dot = sp.diff(z_obj, t)
y_obj_ddot = sp.diff(y_obj_dot, t)
z_obj_ddot = sp.diff(z_obj_dot, t)
obses = [y_obj, z_obj, y_obj_dot, z_obj_dot,
            y_obj_ddot, z_obj_ddot, f_rope_y, f_rope_z]

delta_yh2_global = delta_yh2 * sp.cos(theta2) - delta_zh2 * sp.sin(theta2)
delta_zh2_global = delta_yh2 * sp.sin(theta2) + delta_zh2 * sp.cos(theta2)
f_rope2_y = f_rope2 * sp.sin(theta2+phi2)
f_rope2_z = -f_rope2 * sp.cos(theta2+phi2)
y_hook2 = y2 + delta_yh2_global
z_hook2 = z2 + delta_zh2_global
y_hook2_dot = sp.diff(y_hook2, t)
z_hook2_dot = sp.diff(z_hook2, t)
# obses = [y_obj, z_obj, y_obj_dot, z_obj_dot,
#          y_obj_ddot, z_obj_ddot, f_rope_y, f_rope_z]


# Define inertial reference frame
N = ReferenceFrame("N")
N_origin = Point('N_origin')
A = N.orientnew("A", "Axis", [theta, N.x])
B = A.orientnew("B", "Axis", [phi, A.x])

# Define point
drone = Point("drone")
drone.set_pos(N_origin, y * N.y + z * N.z)
hook = drone.locatenew("hook", delta_yh * A.y + delta_zh * A.z)
drone2 = Point("drone2")
drone2.set_pos(N_origin, y2 * N.y + z2 * N.z)
hook2 = drone2.locatenew("hook2", delta_yh2 * A.y + delta_zh2 * A.z)
obj = hook.locatenew("obj", -l * B.z)
drone.set_vel(N, y_dot * N.y + z_dot * N.z)
drone2.set_vel(N, y2_dot * N.y + z2_dot * N.z)

# Inertia
inertia_quadrotor = inertia(N, I, 0, 0)
quadrotor = RigidBody("quadrotor", drone, A, m, (inertia_quadrotor, drone))
quadrotor2 = RigidBody("quadrotor2", drone2, A, m, (inertia_quadrotor, drone2))
obj_particle = Particle("obj_particle", obj, mo)

# Newton's law
# 8 equations
eq_quad_y = -thrust * sp.sin(theta) + f_rope_y - m * y_ddot
eq_quad_z = thrust * sp.cos(theta) + f_rope_z - m * g - m * z_ddot
eq_quad_theta = tau + delta_yh_global * f_rope_z - \
    delta_zh_global * f_rope_y - I * theta_ddot
# eq_obj_y = -f_rope_y - mo * y_obj_ddot
# eq_obj_z = -f_rope_z - mo * g - mo * z_obj_ddot

eq_quad_y2 = -thrust2 * sp.sin(theta2) + f_rope2_y - m * y2_ddot
eq_quad_z2 = thrust2 * sp.cos(theta2) + f_rope2_z - m * g - m * z2_ddot
eq_quad_theta2 = tau2 + delta_yh2_global * f_rope2_z - \
    delta_zh2_global * f_rope2_y - I * theta2_ddot
eq_obj_y = -f_rope_y -f_rope2_y - mo * y_obj_ddot
eq_obj_z = -f_rope_z -f_rope2_z - mo * g - mo * z_obj_ddot

eq_quad1_obj = (y-y_obj)*(y_dot-y_obj_dot)+(z-z_obj)*(z_dot-z_obj_dot)
eq_quad2_obj = (y2-y_obj)*(y2_dot-y_obj_dot)+(z2-z_obj)*(z2_dot-z_obj_dot)

# TODO...
eqs = [eq_quad_y, eq_quad_z, eq_quad_theta, eq_obj_y, eq_obj_z, eq_quad_y2, eq_quad_z2, eq_quad_theta2, eq_quad1_obj, eq_quad2_obj]
eqs = [eq.expand() for eq in eqs]
eqs = [eq.subs([(states_dot[i], states_dot_val[i])
                for i in range(len(states_dot))]) for eq in eqs]
eqs = [eq.subs([(states[i], states_val[i])
                for i in range(len(states))]) for eq in eqs]
# Solve for the acceleration
A_taut_dyn = sp.zeros(10, 10)
b_taut_dyn = sp.zeros(10, 1)
for i in range(10):
    for j in range(10):
        A_taut_dyn[i, j] = eqs[i].coeff(states_dot_val[j])
    b_taut_dyn[i] = -eqs[i].subs([(states_dot_val[j], 0)
                                    for j in range(10)])
# lambda A_taut_dyn
A_taut_dyn_func = sp.lambdify(
    params + states_val + action, A_taut_dyn, "jax")
b_taut_dyn_func = sp.lambdify(
    params + states_val + action, b_taut_dyn, "jax")
A_taut_dyn


Matrix([
[ -m,   0,                                                                                    0,                            0,                                                                                                                                                                         sin(phi(t) + theta(t)),  0,  0,  0, 0,                                                                                                                                                                                                              0],
[  0,  -m,                                                                                    0,                            0,                                                                                                                                                                        -cos(phi(t) + theta(t)),  0,  0,  0, 0,                                                                                                                   

In [10]:
# import sympy
from sympy import *
x = symbols('x')
expr = sin(x)
# Use sympy.lambdify() method
f = lambdify(x, expr, "math")
print("Using lambda function in SymPy to evaluate sin(90) : {}".format(f(90)))


Using lambda function in SymPy to evaluate sin(90) : 0.8939966636005579
